##I am splitting v3_Clean_model_add_Pis_feat.ipynb into 2 notebooks 

-another one on modelling v1_Model2_Pisch_Eval_Tuning.ipynb

-this one on creating stats data set (just Pisch for now)



In [1]:
import numpy as np
import pandas as pd

###some extra feature stuff


Some simple functions

In [2]:
##have not written yet

def top3_max_val_params(model, X, dates, drop_first=False):
    pass
    


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data

In [3]:
#this is for  regressors predicting wins - losses, can use this to turn output into win prediction 
def fav_win(x):
    if x <=0:
        return 1
    if x>0:
        return 0


def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)

#useage: v_make_win(y_pred)

In [4]:


def one_plus(x):
    return 1+x

v_one_plus = np.vectorize(one_plus)

def minus_one(x):
    return x-1

v_minus_one = np.vectorize(minus_one)

def one_minus(x):
    return 1-x

v_one_minus = np.vectorize(one_minus)


##Some more complex functions useful for generating new 

-basic stats (eg sh%)

-cumulative stats for all prior games for that team up to the present date
(not including present date)

In [5]:
##this function creates the dummy variables for you for evey team ...


##HA_diff does dummies_home - dummies_away
##HA_concat does dummies_home concat dummies_away (to the right)
##! Concat veriosn shouls also do dummy for H/A since it is no linger encoded!
## hmm ... maybe not.. HT_dummies, AT_dummies, HT_stats, AT_stats; Hg-Ag, HTWin 

def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    
    #reset index to prep for df1.sub(df2)
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    
    for feat in ['won', 'goal_difference', 'Open']: ##will go in reverse order 
        df_model.insert(loc=0, column= feat, value=  X_H[feat].copy())
   
    #carefule with home and away teams 
    df_model.insert(loc=0, column= 'away_team', value=  X_A['nhl_name'].copy())
    df_model.insert(loc=0, column= 'home_team', value=  X_H['nhl_name'].copy())
    
    df_model.insert(loc=0, column= 'full_date', value=  X_H['full_date'].copy())

    df_model.insert(loc=0, column= 'game_id', value=  X_H['game_id'].copy())
    
    
    #df_model['home_team'] = X_H['nhl_name'].copy()
    #df_model['away_team'] = X_A['nhl_name'].copy() 
    
    
        #y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return df_model

##try later maye 
def make_HA_concat(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    #df_HA = pd.get_dummies(X['HoA']), dtype=np.int64)
    
    df_model = df_home.sub(df_visitor) 
    
    
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [6]:
['date', 'full_date','game_id', 'Open','goal_difference', 'won' ][::-1]

['won', 'goal_difference', 'Open', 'game_id', 'full_date', 'date']

In [7]:
def make_diff(statFor, statAway):
    return statFor - statAway

v_make_diff = np.vectorize(make_diff)

def make_per(statFor, statAway):  #example FOWFor/(FOWFor + FOWAgainst)  or ShAt
    try: 
        return statFor/(statFor+statAway)
    except: 
        return 0


v_make_per = np.vectorize(make_per)
 
    
def make_ratio(stat1, stat2):  #example goalsFor/shotsFor = sh%
    try:
        return stat1/stat2
    except:
        return 0
    
v_make_ratio = np.vectorize(make_ratio)

##have to adjust later depending what is convenient to use for stat name ... 

#not using k_days_back .. will use a unioversal fn to just avg past games...
#makes more sense ,,, if they get a 1000 shots against one game, don't want that to leak into 
#other games .. 



def get_per(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[statFor+'%' ] = v_make_per(X.loc[:,statFor],X.loc[:,statAgainst]) 
    #return v_make_per(X.loc[:,statFor],X.loc[:,statAgainst])


def get_ratio(X, stat1, stat2, new_stat_name):  #stat1/stat2  
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    
    X[new_stat_name] = v_make_ratio(X.loc[:,stat1],X.loc[:,stat2]) 
    #return v_make_ratio(X.loc[:,stat1],X.loc[:,stat2])

def get_diff(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[stat_name+'Diff']= v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    #return v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    
    

In [8]:
sorted({8,5})

[5, 8]

In [9]:
[2,3,434,45,5,566,6,][-10:]

[2, 3, 434, 45, 5, 566, 6]

In [10]:
##!! you need to loop over nhl_name at the beginning so your dates are associated to that fixed team


#stat_name goalsFor for example
def get_k_game_sum(X, stat_name, k_days_back= 10**6 ): #k_days_back #season is random-default 
   
    #make string version of k_days_back for later; 10**6 means go back forever to beginning of season
    if k_days_back < 10**6:
        str_k = '_'+str(k_days_back)+'_day'
    else: 
        str_k = "_cumul"
    
    #set up column eg goalFor_10_days or 
    X[stat_name+str_k+'_sum'] = np.NaN  
    
     #doing set removes duplicates; sorted makes increasing ordered list 
    all_teams = list(set(X['nhl_name']))   
    
    for nhl_name0 in all_teams:  #I'll label in nahl_name0 to emphasize it is fixed constant
        
        ## this is all the dates *that have this tean nhl_name playing*
        team0_dates  = sorted(set(X.loc[X['nhl_name'] == nhl_name0, 'full_date'])) 
        
        
        #note: first date0 of teh season for the team is special because date < date0 will be empty
        
        for date1 in team0_dates[1:]: #all but first date so don't get empty object with date< date1
            team0_dates_bef_date1 = [date for date in team0_dates if date < date1]  #for the fixed team
            team0_k_dates_bef_date1 = team0_dates_bef_date1[-k_days_back :] # this further restricts to the last k days of list; 
                                                                # or returns all for large k_days_back, [1,2][-10,:] = [1,2]
                                                                #will be nonempty if k_days_back >0
            #we have to restrict to k *team = nhl_name0* games so that's why we do it here 
                    
            #restrict the df to just team0 and dates  < date1 (this is where we get empty if date1 =date0 )
            X_team0_k_bef_date1 = X.loc[(X['nhl_name'] == nhl_name0) & X['full_date'].isin(team0_k_dates_bef_date1), :].copy()           
           
            #main step; first calculate the sum for nhl_name0 and dates < date1 
            k_sum = np.sum(X_team0_k_bef_date1[stat_name])
            
            #here we assign the value to a unique row of the original X which was passed
            #the new columns is eg goalsFor_10_sum or goals_for_cumul_sum
            
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name0) , stat_name+str_k+'_sum'] =k_sum
    
    #you can either operate directly on X which was passed, adding a new column, or you could return a df ... 
    #so far seems convenient to do the former
    #return X

            
            
            #huh? this below is wrong ... date1 should not be touched
            #k_sum = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            #X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+str_k+'_sum'] =k_sum
        

In [11]:
##!! you need to loop over nhl_name at the beginning so your dates are associated to that fixed team


#stat_name goalsFor for example
def get_k_game_avg(X, stat_name, k_days_back= 10**6 ): #k_days_back #season is random-default 
   
    #make string version of k_days_back for later; 10**6 means go back forever to beginning of season
    if k_days_back < 10**6:
        str_k = '_'+str(k_days_back)+'_day'
    else: 
        str_k = "_cumul"
    
    #set up column eg goalFor_10_days or 
    X[stat_name+str_k+'_avg'] = np.NaN  
    
     #doing set removes duplicates; sorted makes increasing ordered list 
    all_teams = list(set(X['nhl_name']))   
    
    for nhl_name0 in all_teams:  #I'll label in nahl_name0 to emphasize it is fixed constant
        
        ## this is all the dates *that have this tean nhl_name playing*
        team0_dates  = sorted(set(X.loc[X['nhl_name'] == nhl_name0, 'full_date'])) 
        
        
        #note: first date0 of teh season for the team is special because date < date0 will be empty
        
        for date1 in team0_dates[1:]: #all but first date so don't get empty object with date< date1
            team0_dates_bef_date1 = [date for date in team0_dates if date < date1]  #for the fixed team
            team0_k_dates_bef_date1 = team0_dates_bef_date1[-k_days_back :] # this further restricts to the last k days of list; 
                                                                # or returns all for large k_days_back, [1,2][-10,:] = [1,2]
                                                                #will be nonempty if k_days_back >0
            #we have to restrict to k *team = nhl_name0* games so that's why we do it here 
                    
            #restrict the df to just team0 and dates  < date1 (this is where we get empty if date1 =date0 )
            X_team0_k_bef_date1 = X.loc[(X['nhl_name'] == nhl_name0) & X['full_date'].isin(team0_k_dates_bef_date1), :].copy()           
           
            #main step; first calculate the sum for nhl_name0 and dates < date1 
            k_avg = np.mean(X_team0_k_bef_date1[stat_name])
            
            #here we assign the value to a unique row of the original X which was passed
            #the new columns is eg goalsFor_10_avg or goals_for_cumul_avg
            
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name0) , stat_name+str_k+'_avg'] =k_avg
    
    #you can either operate directly on X which was passed, adding a new column, or you could return a df ... 
    #so far seems convenient to do the former
    #return X

            
            
            #huh? this below is wrong ... date1 should not be touched
            #k_sum = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            #X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+str_k+'_sum'] =k_sum
        

##Now let's run thru generating the Pisch data set again ... (some corrections)

In [12]:

data = pd.read_csv("/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_bet_stats_mp.csv")
data.drop(columns=[ 'Unnamed: 0'], inplace=True)

In [13]:
data['won'] = data['won'].apply(int)
data_playoffs = data.loc[data['playoffGame'] == 1, :].copy()  #set aside playoff games ... probably won't use them.
data=  data.loc[data['playoffGame'] == 0, :].copy() 
#fix the Nans in FOW%:
data['faceOffTotalBothTeams'] = data['faceOffsWonFor'] + data['faceOffsWonAgainst']
data['faceOffWinPercentage'] = v_make_ratio(data['faceOffsWonFor'],data['faceOffTotalBothTeams'])
 
#sorted(data.columns)

In [14]:
#bad_game_ids with 0 0 score (in df_game_team_stats) ,df_game is probably ok 
bad_game_ids = [2008020057, 2008020071, 2008020306, 2008020623, 2008021108,
            2008021196, 2009020072, 2009020253, 2009020682, 2009020831,
            2009021118, 2009021209, 2010020382, 2010020761, 2010020878,
            2010021111, 2011020749, 2011020787, 2011021016, 2011021052,
            2011021108, 2012020159, 2012020412, 2012020487, 2013020126,
            2013021136, 2013021223, 2014020055, 2014020158, 2014020313,
            2014020456, 2014021008, 2014021210, 2016020785, 2017020561,
            2017020965, 2018020783, 2019020127, 2019021041]

##impute the missing 00 games
data.loc[data['game_id'].isin(bad_game_ids)&(data['won']==1),'goalsFor'] = 1.0
data.loc[data['game_id'].isin(bad_game_ids)&(data['won']==0), 'goalsAgainst'] = 1.0

#verify
#data.loc[data['game_id'].isin(bad_game_ids), ['won', 'goalsFor', 'goalsAgainst']]

In [ ]:
###debugging ... why am I gettin 47% nan in dummies and date, id float?


perc_null(data.loc[data['season'] ==season, :])

season = 20152016

X_seas = data.loc[data['season'] ==season, :].copy()

#Here Pis is for Pischada ... I am following a 2013 paper which he based on the Weissenbock paper (he is at U Ottawa)
#Piscada paper file:///Users/joejohns/Downloads/PredictingNHLmatchoutcomeswithMLmodels%20(1).pdf

feat_Pis = ['goalsAgainst', 'goalsFor', 'goalDiff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

##these ae teh features from X_seas I will need to build the Pisch features

feat_for_Pis_small = [ 'goalsAgainst','goalsFor', 'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst',
                      'shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 'fenwickPercentage',] 

#we start with pp% and pk% below

    get_ratio(X_seas,  'powerPlayGoals','powerPlayOpportunities', 'pp%')

#to do pk% (no SHgoalsAgainst) so we use pk% of HT = 1- pp% of AT and vice versa

#set up the pk% column initialized with 0

    X_seas['pk%'] = 0

##to make this 1-pk% work we make sure the indices are set up consistently (not sure if needed)

    X_seas.sort_values(by = ['full_date', 'game_id', 'HoA'], inplace = True)
    X_seas.reset_index(drop = True, inplace = True)

##note: The following 2 lines did not work when I had X_seas = (...) with no .copy() above!
    X_seas.loc[X_seas['HoA'] == 'home', 'pk%']  = v_one_minus(X_seas.loc[X_seas['HoA'] == 'away' ,['pp%']].copy())
    X_seas.loc[X_seas['HoA'] == 'away', 'pk%'] =  v_one_minus(X_seas.loc[X_seas['HoA'] == 'home' ,['pp%']].copy())


#this creates gfdiff, gf%, sh%, sv%
    get_diff(X_seas, 'goalsFor')
    get_per(X_seas, 'goalsFor')
    get_ratio(X_seas, 'goalsFor', 'shotsOnGoalFor', 'sh%')
    get_ratio(X_seas, 'savedShotsOnGoalAgainst', 'shotsOnGoalAgainst', 'sv%')
#X_seas['sv%'] = one_minus(X_seas['sv%']) don't need this now ... used savedSHA

#pdo is simple sum
    X_seas['PDO'] = X_seas['sh%'] + X_seas['sv%']

##we create a column of 1s to calculated games so far
    X_seas['ones'] = 1
    get_k_game_sum(X_seas, 'ones')
    get_k_game_sum(X_seas, 'won') 

    X_seas.rename(columns = {'ones_cumul_sum': 'team_games_so_far'}, inplace = True)  #counts num of games so far for that team
##total wins and win% (I am ignoring distinction OT/SO/Reg)

    get_ratio(X_seas, 'won_cumul_sum', 'team_games_so_far', 'win%_cumul')

##total wins and win% in the last 10 games ... later can try different versions
    get_k_game_sum(X_seas, 'won', k_days_back=10)
    get_k_game_sum(X_seas, 'ones', k_days_back=10)
    get_ratio(X_seas, 'won_10_day_sum','ones_10_day_sum','win%_last_10_games')


    feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
    feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'PDO','fenwickPercentage', 
'corsiPercentage',  'xGoalsPercentage',   ]  ##added corsi and xgoals 

##NOTE! This is a bit inaccurate to average these ... one should find cumul_sums then do the % calculations above for accurate %
##let's see if they are that different ... 

#'last_10_games_win%',  'win%',     omtted because already a 10 gm avg, 

    for feat_to_sum in feat_Pis_to_sum:
        get_k_game_sum(X_seas, feat_to_sum)
    
    

    for feat_to_avg in feat_Pis_to_avg:
        get_k_game_avg(X_seas, feat_to_avg)
        
    ##get dummies 


    feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 
#did win% and win%_last_10 instead of streak and standing ... did regular fenwick, not fenclose
#might be good to group these into 5-10 groups ... 



    data_Pis_pre_xg_corsi = X_seas.loc[:, ['HoA',
       'goalsAgainst_cumul_sum', 'goalsFor_cumul_sum', 
        'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 
        'sh%_cumul_avg', 'sv%_cumul_avg',
        'PDO_cumul_avg', 'fenwickPercentage_cumul_avg', 'corsiPercentage_cumul_avg',  'xGoalsPercentage_cumul_avg',
        'win%_last_10_games',   'win%_cumul', ]].copy()


In [16]:
df_dic ={}
#set season = 20122013  #select this shortened season because same as Pisch

for season in [20152016, 20162017, 20172018, 20182019]:

    X_seas = data.loc[data['season'] ==season, :].copy()

#Here Pis is for Pischada ... I am following a 2013 paper which he based on the Weissenbock paper (he is at U Ottawa)
#Piscada paper file:///Users/joejohns/Downloads/PredictingNHLmatchoutcomeswithMLmodels%20(1).pdf

    feat_Pis = ['goalsAgainst', 'goalsFor', 'goalDiff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

##these ae teh features from X_seas I will need to build the Pisch features

    feat_for_Pis_small = [ 'goalsAgainst','goalsFor', 'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst',
                      'shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 'fenwickPercentage',] 

#we start with pp% and pk% below

    get_ratio(X_seas,  'powerPlayGoals','powerPlayOpportunities', 'pp%')

#to do pk% (no SHgoalsAgainst) so we use pk% of HT = 1- pp% of AT and vice versa

#set up the pk% column initialized with 0

    X_seas['pk%'] = 0

##to make this 1-pk% work we make sure the indices are set up consistently (not sure if needed)

    X_seas.sort_values(by = ['full_date', 'game_id', 'HoA'], inplace = True)
    X_seas.reset_index(drop = True, inplace = True)

##note: The following 2 lines did not work when I had X_seas = (...) with no .copy() above!
    X_seas.loc[X_seas['HoA'] == 'home', 'pk%']  = v_one_minus(X_seas.loc[X_seas['HoA'] == 'away' ,['pp%']].copy())
    X_seas.loc[X_seas['HoA'] == 'away', 'pk%'] =  v_one_minus(X_seas.loc[X_seas['HoA'] == 'home' ,['pp%']].copy())


#this creates gfdiff, gf%, sh%, sv%
    get_diff(X_seas, 'goalsFor')
    get_per(X_seas, 'goalsFor')
    get_ratio(X_seas, 'goalsFor', 'shotsOnGoalFor', 'sh%')
    get_ratio(X_seas, 'savedShotsOnGoalAgainst', 'shotsOnGoalAgainst', 'sv%')
#X_seas['sv%'] = one_minus(X_seas['sv%']) don't need this now ... used savedSHA

#pdo is simple sum
    X_seas['PDO'] = X_seas['sh%'] + X_seas['sv%']

##we create a column of 1s to calculated games so far
    X_seas['ones'] = 1
    get_k_game_sum(X_seas, 'ones')
    get_k_game_sum(X_seas, 'won') 

    X_seas.rename(columns = {'ones_cumul_sum': 'team_games_so_far'}, inplace = True)  #counts num of games so far for that team
##total wins and win% (I am ignoring distinction OT/SO/Reg)

    get_ratio(X_seas, 'won_cumul_sum', 'team_games_so_far', 'win%_cumul')

##total wins and win% in the last 10 games ... later can try different versions
    get_k_game_sum(X_seas, 'won', k_days_back=10)
    get_k_game_sum(X_seas, 'ones', k_days_back=10)
    get_ratio(X_seas, 'won_10_day_sum','ones_10_day_sum','win%_last_10_games')


    feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
    feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'PDO','fenwickPercentage', 
'corsiPercentage',  'xGoalsPercentage',   ]  ##added corsi and xgoals 

##NOTE! This is a bit inaccurate to average these ... one should find cumul_sums then do the % calculations above for accurate %
##let's see if they are that different ... 

#'last_10_games_win%',  'win%',     omtted because already a 10 gm avg, 

    for feat_to_sum in feat_Pis_to_sum:
        get_k_game_sum(X_seas, feat_to_sum)
    
    

    for feat_to_avg in feat_Pis_to_avg:
        get_k_game_avg(X_seas, feat_to_avg)
        
    ##get dummies 


    feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 
#did win% and win%_last_10 instead of streak and standing ... did regular fenwick, not fenclose
#might be good to group these into 5-10 groups ... 



    data_Pis_pre_xg_corsi = X_seas.loc[:, ['HoA',
       'goalsAgainst_cumul_sum', 'goalsFor_cumul_sum', 
        'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 
        'sh%_cumul_avg', 'sv%_cumul_avg',
        'PDO_cumul_avg', 'fenwickPercentage_cumul_avg', 'corsiPercentage_cumul_avg',  'xGoalsPercentage_cumul_avg',
        'win%_last_10_games',   'win%_cumul', ]].copy()
    

    data_Pis_pre = X_seas.loc[:, ['HoA',
       'goalsAgainst_cumul_sum', 'goalsFor_cumul_sum', 
        'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 
        'sh%_cumul_avg', 'sv%_cumul_avg',
        'PDO_cumul_avg', 'fenwickPercentage_cumul_avg', 
        'win%_last_10_games',   'win%_cumul', ]].copy()

    
    #df_dic["data_Pis_pre_xg_corsi_"+str(season)] = data_Pis_pre_xg_corsi
    #df_dic["data_Pis_pre_"+str(season)] = data_Pis_pre
#targets   
#  'won', 'goal_difference' ,'goalsAgainst','goalsFor', 'Open'
#id stuff
##'full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam', 

    data_Pis_H = data_Pis_pre.loc[data_Pis_pre['HoA'] =='home',:].copy()
    data_Pis_A = data_Pis_pre.loc[data_Pis_pre['HoA'] =='away',:].copy()

#reset index for df1.sub(df2)
    data_Pis_A.reset_index(drop = True, inplace = True)
    data_Pis_H.reset_index(drop = True, inplace = True)

##set the numerical data to home stats - away stats 
    data_Pis = data_Pis_H.iloc[:, 1:].copy().sub(data_Pis_A.iloc[:, 1:].copy()).copy()   #remove the 'HoA' column with 1:


    dummies_pm1_Pis = make_HA_diff(data, season = season).copy()  #single df, has id stuff and target stuff!
    dummies_pm1_Pis.reset_index(drop = True, inplace =True)

#combine dummies and data 
    data_dummies_Pis = pd.concat([dummies_pm1_Pis, data_Pis], axis =1)
    
    df_dic["data_Pis_pre_xg_corsi_"+str(season)] = data_dummies_Pis
    
    filename_seas = 'data_dummies_Pis_xg_Corsi_v3_'+str(season)+'.csv'
    data_dummies_Pis.to_csv(filename_seas)







##moved into for loop over seasons ...

feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'fenwickPercentage', 'PDO',    ]  

##NOTE! This is a bit inaccurate to average these ... one should find cumul_sums then do the % calculations above for accurate %
##let's see if they are that different ... 

#'last_10_games_win%',  'win%',     omtted because already a 10 gm avg, 

for feat_to_sum in feat_Pis_to_sum:
    get_k_game_sum(X_12, feat_to_sum)
    
    

for feat_to_avg in feat_Pis_to_avg:
    get_k_game_avg(X_12, feat_to_avg)





In [ ]:
# check stuff 'SJS', 'ANA' ... looks good

##chnage X_12 to X_20162017 = pd.read_csv(...)
#X_12.loc[X_12['nhl_name'] == 'SJS', ['won','nhl_name', 'team_games_so_far','full_date', 'sv%', 'sv%_cumul_avg', 'win%',  'won_10_day_sum','ones_10_day_sum','last_10_games_win%']]

In [ ]:
##note: restrict the dates later if you want to muchacho
##and remove the target etc at modelling time
#data_dummies_Pis.iloc[20:50,:]

In [ ]:
##fiddle with this later


feat_mine = [ 'full_date',
'season',
 'game_id',
'HoA',
'nhl_name',
 'opposingTeam',
'settled_in',
'playoffGame',
          'situation', 
             
    'won',   
'goalsAgainst',
 'goalsFor',
             
 'penalityMinutesAgainst',
 'penalityMinutesFor',
 'penaltiesAgainst',
 'penaltiesFor',
'powerPlayGoals',
 'powerPlayOpportunities',
             
'faceOffsWonAgainst',
 'faceOffsWonFor',
'faceOffWinPercentage', #filled in
             
'giveawaysAgainst',
 'giveawaysFor',
'dZoneGiveawaysAgainst',
 'dZoneGiveawaysFor',
    
 
 'shotAttemptsAgainst',
 'shotAttemptsFor',
'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor',
 'shotsOnGoalAgainst',
 'shotsOnGoalFor',
 'savedShotsOnGoalAgainst',
 'savedShotsOnGoalFor',
 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
             
 'xFreezeAgainst',
 'xFreezeFor',
 'xGoalsAgainst',
 'xGoalsFor',
 
'scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor',
 
'xGoalsPercentage',
 'corsiPercentage', 

'fenwickPercentage',
             
    'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
    'highDangerxGoalsAgainst',
 'highDangerxGoalsFor',
'highDangerShotsAgainst',
 'highDangerShotsFor',]

sorted(data.columns)

In [ ]:
data['faceOffWinPercentage'].isnull().value_counts()

In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_Pis_plus = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

#features I need to do this ...
feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
#'win_streak_grouped_10', 'conference_standing_grouped_10',
#'PDO'  

#targets 
 #could later try to train classifier to predict this ... tough tho --> reg or no
    

extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]



feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
###set Leung aside for now ... 
    
feat_Leung = ['ID', 'Date', 'HomeTeam', 'AwayTeam', 'GDiff','GF%', 'CF%','CSh%',
'CSv%', 'FF%','FSh%','FSv%','PDO','PENDiff','ShF%','SDiff','Sh%', 'Sv%','FOW%','W%','FavoritesW%', 'Result']


feat_for_Leung  = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]
    
    
feat_plus = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%',  ]


##checking the calculations done to make the data set 

Overall, the sums, differences, ratios, cumulative sums and cumulative avgs 
are working well! 

what I checked: (beginning of season for SJS and a little bit TBL, I  eye-balled and also checked a few with a calculator; end of season just eye-balled a few)

The following problems were found:  [now fixed as of 7pm Aug 6]

1. df_game_team_stats has around 40 games with 0 0 score (df_game is ok)
note: they are all OT 1-0 games (from df_game) so these can be imputed using the "won" column (the game_ids are below)

2. get_k_avg and get_k_sum need to loop over the team so that the dates are restricted to the fixed team (rather than all the last 10 dates regardless of whether the team played or not). This error messed up: wins in last 10 and stuff (counting ones also)

3. the first game of teams starting after first game of season have a 
bad first row ... it should be NaN but instead it is same as game 2 ... after that no errors game 2 on ... so not big deal. Probably date = date0
etc. is messed up in get_k_sum and get_k_avg ... probably looping over teams first will fix it like in 2.




In [ ]:
##check stats calculations first few games ... 
##part 1 ... goals for and against stats ... 
##they are all OT 1-0 wins ... so you can impute this using the "won" column

X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won', 'settled_in','games_so_far', 'goalsAgainst',  'goalsFor', 'goalsFor%', 'goalsDiff','goalsDiff_cumul_sum','goalsAgainst_cumul_sum',   'goalsFor_cumul_sum',    'goalsFor%_cumul_avg', 
   ]].iloc[:15, :]

In [ ]:
'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage',    'sh%_cumul_avg', 'sv%_cumul_avg', 'PDO',  'PDO_cumul_avg'
       'fenwickPercentage_cumul_avg',
     'sh%', 'sv%', 
      'powerPlayGoals', 'powerPlayOpportunities',
            'pp%', 'pk%', 'pp%_cumul_avg', 'pk%_cumul_avg',
       

In [ ]:
##check stats calculations first few games ... 
##part 4 ...shots, fenwick 
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'powerPlayGoals', 'powerPlayOpportunities',
            'pp%', 'pk%', 'pp%_cumul_avg', 'pk%_cumul_avg', ]].iloc[:20, :]

In [ ]:
##check stats calculations first few games ... 
##part 3 ... pp, 
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst', 
       'savedShotsOnGoalFor', 'sh%', 'sv%','PDO', 'PDO_cumul_avg','sh%_cumul_avg', 'sv%_cumul_avg', 'fenwickPercentage',     
       'fenwickPercentage_cumul_avg',
      ]].iloc[-8:, :]

In [ ]:
##check stats calculations first few games ... 
##part 2 ... wins, win% ...
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'won_cumul_sum', 'games_so_far', 'win%', 'ones','won_10_day_sum',
       'ones_10_day_sum', 'last_10_games_win%', ]].iloc[:20, :]